In [ ]:
# Premonitor: mock_hardware.py
# This script simulates the hardware sensors for the Premonitor project.
# It allows for full end-to-end testing of the software and AI models on the
# Raspberry Pi without needing the physical sensors to be connected.
# It reads sample data from the downloaded public datasets.
#What Replaces the Mock Hardware?
#You asked an excellent question about what you'll use when you have the real hardware. Once you've purchased the sensors, you will create a new file, let's call it hardware_drivers.py, that will have the exact same function names as this mock file (read_thermal_image, read_acoustic_spectrogram, etc.).
#This new file will use specific Python libraries to talk to the real hardware:
#To replace read_thermal_image(): You will use a library specific to your thermal camera. For example, if you buy a FLIR Lepton camera with a PureThermal board, you might use the pylepton library to capture a real thermal image array from the camera's USB port.
#To replace read_acoustic_spectrogram(): You will use a library like pyaudio or sounddevice to capture a few seconds of audio from the real USB microphone. Your function will then pass this audio chunk to the utils.audio_to_spectrogram() function you've already written.
#To replace read_gas_sensor(): You will use the RPi.GPIO library or Adafruit-Blinka with a library for an ADC (Analog-to-Digital Converter) chip like the MCP3008. This will allow you to read the analog voltage from the real MQ gas sensor connected to the Pi's GPIO pins.
#The beauty of this approach is that when you're ready to switch to real hardware, you will only need to change one line in your main.py file: from import mock_hardware as hardware to import hardware_drivers as hardware. The rest of your application will work exactly the same.

import os
import random
import numpy as np
import tensorflow as tf

# Import our custom project configuration
import config
import utils # We need this for the audio_to_spectrogram function

# --- Global variables to hold the paths to our mock data ---
MOCK_THERMAL_IMAGE_PATHS = []
MOCK_ACOUSTIC_FILE_PATHS = []

def initialize_mock_data():
    """
    Scans the dataset directories and populates the file path lists.
    This should be called once when the main application starts.
    """
    global MOCK_THERMAL_IMAGE_PATHS, MOCK_ACOUSTIC_FILE_PATHS
    print("Initializing mock hardware: loading data file paths...")

    # --- Load Thermal Image Paths (using FLIR dataset as an example) ---
    # NOTE: Update this path to where you have saved the FLIR dataset
    thermal_data_dir = 'datasets/FLIR_ADAS_v2/images_thermal_train/data'
    if os.path.exists(thermal_data_dir):
        MOCK_THERMAL_IMAGE_PATHS = [os.path.join(thermal_data_dir, f) for f in os.listdir(thermal_data_dir) if f.endswith('.jpeg')]
        print(f"Found {len(MOCK_THERMAL_IMAGE_PATHS)} mock thermal images.")
    else:
        print(f"Warning: Mock thermal data directory not found at '{thermal_data_dir}'. The mock function will return empty data.")

    # --- Load Acoustic File Paths (using MIMII fan dataset as an example) ---
    # NOTE: Update this path to where you have saved the MIMII dataset
    # We will use both normal and abnormal sounds to test our model's response
    acoustic_normal_dir = 'datasets/MIMII/0_dB_fan/train/normal'
    acoustic_abnormal_dir = 'datasets/MIMII/0_dB_fan/test/abnormal'

    if os.path.exists(acoustic_normal_dir):
        MOCK_ACOUSTIC_FILE_PATHS.extend([os.path.join(acoustic_normal_dir, f) for f in os.listdir(acoustic_normal_dir) if f.endswith('.wav')])

    if os.path.exists(acoustic_abnormal_dir):
        MOCK_ACOUSTIC_FILE_PATHS.extend([os.path.join(acoustic_abnormal_dir, f) for f in os.listdir(acoustic_abnormal_dir) if f.endswith('.wav')])

    if MOCK_ACOUSTIC_FILE_PATHS:
        print(f"Found {len(MOCK_ACOUSTIC_FILE_PATHS)} mock acoustic files.")
    else:
        print(f"Warning: Mock acoustic data directories not found. The mock function will return empty data.")

    print("Mock hardware initialization complete.")

def read_thermal_image():
    """
    Simulates reading from a thermal camera by returning a random image
    from the pre-loaded dataset paths.

    Returns:
        A NumPy array representing an image, in the format expected by the AI model.
    """
    if not MOCK_THERMAL_IMAGE_PATHS:
        if config.DEBUG_MODE:
            print("MOCK_THERMAL: No image paths loaded. Returning empty array.")
        return np.zeros(config.THERMAL_MODEL_INPUT_SHAPE, dtype=np.float32)

    random_image_path = random.choice(MOCK_THERMAL_IMAGE_PATHS)

    if config.DEBUG_MODE:
        print(f"MOCK_THERMAL: Reading mock image from {os.path.basename(random_image_path)}")

    # Use the loader from utils to ensure consistent pre-processing
    # We need to load the image and then get its numpy value for the mock hardware
    image_tensor, _ = utils.load_image_and_label(random_image_path, 0) # Label is irrelevant here

    return image_tensor.numpy() if image_tensor is not None else np.zeros(config.THERMAL_MODEL_INPUT_SHAPE, dtype=np.float32)


def read_acoustic_spectrogram():
    """
    Simulates reading from a microphone by selecting a random audio file
    and converting it to a spectrogram.

    Returns:
        A NumPy array representing a spectrogram, ready for the acoustic model.
    """
    if not MOCK_ACOUSTIC_FILE_PATHS:
        if config.DEBUG_MODE:
            print("MOCK_ACOUSTIC: No audio paths loaded. Returning empty array.")
        return np.zeros(config.ACOUSTIC_MODEL_INPUT_SHAPE, dtype=np.float32)

    random_audio_path = random.choice(MOCK_ACOUSTIC_FILE_PATHS)

    if config.DEBUG_MODE:
        print(f"MOCK_ACOUSTIC: Reading mock audio from {os.path.basename(random_audio_path)}")

    # Use the utility function to perform the conversion
    spectrogram = utils.audio_to_spectrogram(random_audio_path)

    if spectrogram is not None:
        # Resize to the model's expected input shape and add a channel dimension
        spectrogram_resized = tf.image.resize(spectrogram[..., np.newaxis], config.ACOUSTIC_MODEL_INPUT_SHAPE[:2])
        return spectrogram_resized.numpy()
    else:
        return np.zeros(config.ACOUSTIC_MODEL_INPUT_SHAPE, dtype=np.float32)

def read_gas_sensor():
    """
    Simulates reading from an MQ-series gas sensor.
    It will usually return a low "normal" value, but has a small chance
    of returning a high "anomaly" value to test the alerting logic.
    """
    # 95% chance of being normal, 5% chance of being an anomaly
    if random.random() < 0.95:
        # Return a normal reading
        reading = random.randint(200, 350)
    else:
        # Return an anomalous reading
        reading = random.randint(700, 900)
        if config.DEBUG_MODE:
            print("MOCK_GAS: Injecting anomalous reading!")

    if config.DEBUG_MODE:
        print(f"MOCK_GAS: Reading is {reading}")

    return reading

# This allows you to run the file directly to test the mock functions
if __name__ == '__main__':
    print("--- Running Mock Hardware Checks ---")
    initialize_mock_data()

    print("\nTesting thermal reader...")
    thermal_data = read_thermal_image()
    print(f"Returned thermal data with shape: {thermal_data.shape} and type: {thermal_data.dtype}")

    print("\nTesting acoustic reader...")
    acoustic_data = read_acoustic_spectrogram()
    print(f"Returned acoustic data with shape: {acoustic_data.shape} and type: {acoustic_data.dtype}")

    print("\nTesting gas sensor reader (run a few times)...")
    for _ in range(5):
        read_gas_sensor()